In [49]:
import pandas_gbq
import pandas as pd
from google.cloud import bigquery

In [ ]:
def load_bway_data(df,proj_id,dest_table):
    # first remove the season/wk data that exists in dest_table in order to not insert duplicated data for the week
    remove_season_weeks_if_exist(df,proj_id,dest_table)

    # now load df to dest_table
    load_to_gbq(df,proj_id,dest_table)

In [51]:
def load_to_gbq(df,proj_id,dest_table):
    # load df to dest_table    
    pandas_gbq.to_gbq(dataframe=df, project_id=proj_id, destination_table = dest_table,if_exists='append')

In [52]:
def remove_season_weeks_if_exist(df,proj_id,dest_table):
    # first remove any season/week_num that exist in df from dest_table
    #   - we don't want to upload duplicate data for the same week
    df_season_wks = df[['season', 'week_num']].drop_duplicates()

    condition = ''

    # build the query condition for which rows need to be removed from dest_table
    for index,row in df_season_wks.iterrows():
        if index!=0:
            condition += "or "
        condition += f'''(season='{row['season']}' and week_num={row['week_num']})\n'''

    qry = f'''
        CREATE OR REPLACE TABLE {proj_id}.{dest_table} AS
        SELECT *
        from {proj_id}.{dest_table}
        where not ( {condition} )
    '''

    # set up the client
    client = bigquery.Client()

    # Run the query to remove dest_table rows and get the result
    query_job = client.query(qry)
    return query_job.result()